In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

#we will be dealing with all images in a standard size (128,128)

In [ ]:
import os

def load_dataset(data_path):
    
    transformation = transforms.Compose([
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    full_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transformation
    )
    
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=50,
        num_workers=0,
        shuffle=False
    )
    
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=50,
        num_workers=0,
        shuffle=False
    )
        
    return train_loader, test_loader

#directory heirarchy is as:
#train
#---Dog
#---Cat
training_folder_name = '../train'

classes = sorted(os.listdir(training_folder_name))
print(classes)
train_loader, test_loader = load_dataset(training_folder_name)
batch_size = train_loader.batch_size

In [ ]:
class Net(nn.Module):
  
    def __init__(self, num_classes=3):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.drop = nn.Dropout2d(p=0.3)
        self.fc = nn.Linear(in_features=32 * 32 * 24, out_features=num_classes)

    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.dropout(self.drop(x), training=self.training)
        x = x.view(-1, 32 * 32 * 24)
        x = self.fc(x)
        return torch.log_softmax(x, dim=1)
    
model = Net(num_classes=len(classes))

print(model)

In [ ]:
def train(model, train_loader, optimizer, epoch):
    
    model.train()
    train_loss = 0
    print("Epoch:", epoch)
    for batch_idx, (data, target) in enumerate(train_loader):
      
        optimizer.zero_grad()
        output = model(data)
        loss = loss_criteria(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        print('Training batch {} Loss = {}'.format(batch_idx + 1, loss.item()))
            
    avg_loss = train_loss / (batch_idx+1)
    print('Training set: Average loss: {}'.format(avg_loss))
    return avg_loss
            
            
def test(model, test_loader):
   
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        batch_count = 0
        for data, target in test_loader:
            batch_count += 1
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_criteria(output, target).item()
            _val, predicted = torch.max(output.data, 1)
            correct += torch.sum(target==predicted).item()

    avg_loss = test_loss/batch_count
    print('Validation set: Average loss: {}, Accuracy: {}/{} ({}%)\n'.format(
        avg_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return avg_loss
    
    
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_criteria = nn.CrossEntropyLoss()

epoch_nums = []
training_loss = []
validation_loss = []

epochs = 10
for epoch in range(1, epochs + 1):
        train_loss = train(model, train_loader, optimizer, epoch)
        test_loss = test(model, test_loader)
        epoch_nums.append(epoch)
        training_loss.append(train_loss)
        validation_loss.append(test_loss)

In [ ]:
from PIL import Image
def resize_image(src_img, size=(128,128), bg_color="white"): 
    
    src_img.thumbnail(size, Image.ANTIALIAS)
    new_image = Image.new("RGB", size, bg_color)
    new_image.paste(src_img, (int((size[0] - src_img.size[0]) / 2), int((size[1] - src_img.size[1]) / 2)))
    return new_image

def predict_image(classifier, image_array):

    classifier.eval()
    class_names = ['Dog','Çat']
    transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    image_tensor = torch.stack([transformation(image).float() for image in image_array])
    input_features = image_tensor
    predictions = classifier(input_features)
    
    predicted_classes = []
    for prediction in predictions.data.numpy():
        class_idx = np.argmax(prediction)
        predicted_classes.append(class_names[class_idx])
    return np.array(predicted_classes)


In [ ]:
#this piece of code shows the prediction for a few images along with the image itself

import os
from random import randint
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

test_folder = '../test'
test_image_files = os.listdir(test_folder)

image_arrays = []

size = (128,128)
background_color="white"

fig = plt.figure(figsize=(12, 8))
for file_idx in range(len(test_image_files)):
    img = Image.open(os.path.join(test_folder, test_image_files[file_idx]))
    resized_img = np.array(resize_image(img, size, background_color))
    image_arrays.append(resized_img)

predictions = predict_image(model, np.array(image_arrays))

for idx in range(len(predictions)):
    a=fig.add_subplot(1,len(predictions),idx+1)
    imgplot = plt.imshow(image_arrays[idx])
    a.set_title(predictions[idx])
